In [1]:
%tensorflow_version 1.x
from google.colab import drive
drive.mount('/content/drive',force_remount = False)
import os
os.chdir('/content/drive/Shareddrives/pzy970802@gmail.com')
# !pip install keras==2.3.1
import sys
sys.path.append('/content/drive/Shareddrives/pzy970802@gmail.com/2021/')
import warnings
warnings.filterwarnings("ignore")

import random
import numpy as np
import sys
sys.path.append("/content/drive/Shareddrives/pzy970802@gmail.com/2021")
from bert4keras.backend import keras, set_gelu, K
from bert4keras.tokenizers import Tokenizer
from bert4keras.models import build_transformer_model
from bert4keras.optimizers import Adam, AdaFactorV1
from bert4keras.snippets import sequence_padding, DataGenerator, text_segmentate
from bert4keras.snippets import text_segmentate, to_array
from keras.layers import Dropout, Dense, Lambda, LSTM, GlobalAveragePooling1D
from sklearn.metrics import f1_score 
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import tensorflow as tf
import os
os.chdir('/content/drive/Shareddrives/pzy970802@gmail.com')

set_gelu('tanh')  # 切换gelu版本

#small数据集里共有52601个id，1600个fandoms, 52655个作者
#按7:3的比例分成train/dev数据集，train里不同的作者有38083, dev不同的作者有18202

import json
from tqdm import tqdm
import numpy as np


import re
valid_truth_dir='/content/drive/Shareddrives/pzy970802@gmail.com/Pzeyang/202104/n_authorship-verification/输出result文件/truth.json'
fw = open(valid_truth_dir,'w')
dir = '/content/drive/Shareddrives/MirthChen1101@gmail.com/Zeyang Peng/authorship-verification/n_small_data.jsonl'
with open(dir,'r') as f:
    # train_text=[]
    # valid_text=[]
    text=[]
    for l in tqdm(f):
        data = json.loads(l)
        text1 = text_segmentate(data['pair'][0], maxlen=510,seps='.?!')
        text2 = text_segmentate(data['pair'][1], maxlen=510,seps='.?!')

        while len(text1)<30 or len(text2)<30:
            if len(text1)<30:
                n_text1=[]
                for i in range(30):
                    for sent in text1:
                        n_text1.append(sent)
                text1=n_text1
            elif len(text2)<30:
                n_text2=[]
                for i in range(30):
                    for sent in text2:
                        n_text2.append(sent)
                text2=n_text2
        text.append((text1, text2, int(data['same']), str(data['id']), data['same'], data['authors']))

print(text[0])
np.random.seed(1)
np.random.shuffle(text)
print(text[0])

TensorFlow 1.x selected.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Using TensorFlow backend.
52601it [00:40, 1305.38it/s]

(['I shift a bit, warily letting my eyes dart from one owl to the other -- but my eyes are trained on the Barn Owl the most. Like Hoole...so like Hoole... He turns a bit, and our eyes meet directly. I can"t describe it...in this next moment, I don"t look away, how awkward it seems. I stare into his eyes. They"re like Hoole"s... They are Barn Owl eyes, but Hoole"s eyes. They"re his eyes...Hoole"s eyes...', 'They hold that light of valor, justice, that one glow that I always made me feel my gizzard twitch in the bottom of my heart. Hoole... He never wanted me, did he? He loved me, but only who I was pretending to be. He never loved me, only Emerilla...lucky owl. I just wanted to belong -- because I was nothing -- and he gave me that. He was so fair, so kind...he gave me something even Kreeth could not. Mum and Da -- Ygryk and Pleek -- were supposed to be my parents. I was supposed to be their chick, Lutta.', 'Yet they left me. They talked about me behind my back...I was a freak. My own p

In [2]:

maxlen = 256
batch_size = 1

config_path = 'dataset/cased_L-12_H-768_A-12/bert_config.json' 
checkpoint_path = 'dataset/cased_L-12_H-768_A-12/bert_model.ckpt'
dict_path = 'dataset/cased_L-12_H-768_A-12/vocab.txt'

 # 建立分词器
tokenizer = Tokenizer(dict_path, do_lower_case=False)

class data_generator(DataGenerator):
    """数据生成器
    """
    def __iter__(self, random=False):
        batch_token_ids, batch_segment_ids, batch_labels = [], [], []
        for is_end, (text1, text2, label, id, _, _) in self.sample(random):
            for index, sent in enumerate(text1[:30]):
                token_ids, segment_ids = tokenizer.encode(text1[index], text2[index], maxlen=maxlen)
                batch_token_ids.append(token_ids)
                batch_segment_ids.append(segment_ids)
                batch_labels.append([label])
                if len(batch_labels)//30 == self.batch_size or is_end:
                    batch_token_ids = sequence_padding(batch_token_ids)
                    batch_segment_ids = sequence_padding(batch_segment_ids)
                    batch_labels = sequence_padding(batch_labels)
                    # prediction=medi_layer.predict([batch_token_ids, batch_segment_ids],batch_size=800,use_multiprocessing=True)

                    # yield prediction, batch_labels
                    yield [batch_token_ids, batch_segment_ids], batch_labels
                    batch_token_ids, batch_segment_ids, batch_labels = [], [], []
                    # prediction = tf.zeros(shape=(30,768))

# 转换数据集
train_generator = data_generator(text[:42000], batch_size)
valid_generator = data_generator(text[42000:], batch_size)

In [3]:

def evaluate(data):
    total, right = 0., 0.
    pred = []
    true = []
    for x_true, y_true in data:
        y_pred = model.predict(x_true).argmax(axis=1)
        pred.append(y_pred)
        y_true = y_true[:, 0]
        total += len(y_true)
        right += (y_true == y_pred).sum()
        true.extend(y_true)
    y_true=true
    y_pred=[]
    for i in range(len(pred)):
        for j in pred[i]:
            y_pred.append(j)
    
    f1=f1_score(y_true, y_pred)
    return right / total, f1

class Evaluator(keras.callbacks.Callback):
    """评估与保存
    """
    def __init__(self):
        self.best_val_acc = 0.

    def on_epoch_end(self, epoch, logs=None):

        val_acc, val_f1 = evaluate(valid_generator)
        if val_acc > self.best_val_acc:
            self.best_val_acc = val_acc
            model.save_weights('/content/drive/Shareddrives/pzy970802@gmail.com/Pzeyang/202105/authorship-verification/small_seed1_1_256/'+str(epoch)+'微调45000_maxlen256_30.weights')
        print(
            u'val_acc: %.5f, best_val_acc: %.5f, val_f1_socre: %.5f\n' %
            (val_acc, self.best_val_acc, val_f1)
        )

evaluator = Evaluator()

### 微调bert模型保存权重

In [ ]:
# 加载预训练模型
bert0 = build_transformer_model(
    config_path=config_path,
    checkpoint_path=checkpoint_path,
    # with_pool=True,
    return_keras_model=False,
    num_hidden_layers=12,
)

output1 = Lambda(lambda x: x[:, 0])(bert0.model.output)

output = Dropout(rate=0.2)(output1)
# output = Dense(units=52655, activation='softmax', )(output)
output = Dense(units=2, activation='softmax', )(output)

model = keras.models.Model(bert0.model.inputs, output)
# model.load_weights('/content/drive/Shareddrives/pzy970802@gmail.com/Pzeyang/202105/authorship-verification/small_seed1_1_256/1128_text4_36820.weights')
model.compile(
                optimizer=Adam(2e-5),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy']
              )
model.summary()
model.fit(
    train_generator.forfit(),
    steps_per_epoch=len(train_generator),
    epochs=4,
    callbacks=[evaluator]
)



Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, None)         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     (None, None, 768)    22268928    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, None, 768)    1536        Input-Segment[0][0]              
___________

### 加载上面保存好的.weights，生成文档的句子表示

In [ ]:
# 加载预训练模型
bert0 = build_transformer_model(
    config_path=config_path,
    checkpoint_path=checkpoint_path,
    # with_pool=True,
    return_keras_model=False,
    num_hidden_layers=12,
)

output1 = Lambda(lambda x: x[:, 0])(bert0.model.output)

output = Dropout(rate=0.2)(output1)
# output = Dense(units=52655, activation='softmax', )(output)
output = Dense(units=2, activation='softmax', )(output)

model = keras.models.Model(bert0.model.inputs, output)
model.load_weights('/content/drive/Shareddrives/pzy970802@gmail.com/Pzeyang/202105/authorship-verification/small_seed1_1_256/1微调45000_maxlen256_30.weights')

from keras.models import load_model, Model

cls_layer = Model(inputs=model.input, outputs=model.layers[-3].output)

def get_sents_represent(generator, texts_num):
    sents_r = np.array([])
    for x_true, y_true in tqdm(generator):
        sent_r = cls_layer.predict(x_true)
        sents_r = np.append(sents_r, sent_r)

    sents_rs = sents_r.reshape(texts_num, 30, 768)

    return sents_rs

train_sents_r = get_sents_represent(train_generator, len(text[:42000]))
valid_sents_r = get_sents_represent(valid_generator, len(text[42000:]))

labels=[]
for i in text:
    labels.append(i[2])

train_labels=np.array(labels[:42000])
valid_labels=np.array(labels[42000:])

### 将得到的句子表示接个神经网络训练

In [ ]:
from keras.layers import Input,Embedding,LSTM,Dense,Flatten,GlobalAveragePooling1D
from keras.models import Model, Sequential
from keras import backend as K

model = Sequential()
# model.add(Flatten())
model.add(GlobalAveragePooling1D())
model.add(Dropout(rate=0.2))
model.add(Dense(16, activation='relu', ))
model.add(Dense(units=2, activation='softmax', ))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(train_sents_r,train_labels,epochs=500,validation_data=(valid_sents_r,valid_labels))
model.save('/content/drive/Shareddrives/pzy970802@gmail.com/Pzeyang/202104/n_authorship-verification/输出result文件/final_微调45000_maxlen256_30.h5')